I'll be showing how the HSAR userclass works and comparing the results from OLS and the HSAR in estimating effects for the example data provided.

In [19]:
import hlm
import pysal as ps
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('./hlm/dong_harris/tests/test.csv')

In [6]:
data.head()

,id,u.full,y,x,county
0,0,-0.689048,0.788457,1,1
1,1,-0.689048,0.788457,0,1
2,2,-0.689048,1.064711,0,1
3,3,-0.689048,0.000000,0,1
4,4,-0.847313,1.131402,0,2


In [8]:
y = data[['y']].values
X = data[['x']].values

In [9]:
W = ps.open('./hlm/dong_harris/tests/w_lower.mtx').read()
M = ps.open('./hlm/dong_harris/tests/w_upper.mtx').read()
W.transform = M.transform = 'r'

In [10]:
membership = data[['county']].values - 1

In [11]:
ols = ps.spreg.OLS(y,X,W)

In [24]:
pd.DataFrame(np.hstack((ols.betas, ols.std_err.reshape(ols.betas.shape))),
             columns=['$\\beta$', '$\sigma_\\beta$'])

,$\beta$,$\sigma_\beta$
0,1.326744,0.029721
1,-0.613395,0.072841


Thus, if we just estimate these naively via ols, this is what we get

### The Dong Harris HSAR

The user class will take 1000 samples if you don't pass a `cycles` argument. So, just to show how the user class works, I'll set it up but not start the sampler.

In [26]:
dh = hlm.HSAR(y,X,W,M,membership=membership, cycles=0)

The public methods and attributes in the sampler are:

In [28]:
[x for x in dir(dh) if not x.startswith('_')]

['M',
 'W',
 'current',
 'cycle',
 'cycles',
 'front',
 'hypers',
 'position',
 'previous',
 'sample',
 'samplers',
 'step',
 'steps',
 'trace',
 'var_names']

#### Defining a sampler

A Gibbs sampler is composed of a list of $p$ distributions $P(.|\mathcal{S_t})$, where $\mathcal{S_t}$ is the internal state of the sampler.

each $P(.|S_t)$ computes the value of its attached parameter using the current values of the parameters stored in the state. For example, if you have a model with three parameters, $\alpha$, $\beta$, $\theta$, where 

$$\alpha \sim \mathcal{N}(.5 * \beta, \theta)$$
$$\beta \sim \mathcal{G}(\alpha, \theta)$$
$$\theta \sim \mathcal{IG}(\alpha/2, \beta)$$

then the Gibbs sampler describing that setup would use the conditional distributions:
$$ P(\alpha_{t+1} | \beta_t, \theta_t)$$
$$ P(\beta_{t+1} | \alpha_{t+1}, \theta_t)$$
$$ P(\theta_{t+1} | \alpha_{t+1}, \beta_{t+1})$$

The ordered set of these distributions is called $\mathcal{P}$.  Also, let us say there are $p$ elements in $\mathcal{P}$. The order of elements in $\mathcal{P}$ is arbitrary, in theory, but any implementation will likely structure the order to help memoize certain expensive computations. 

#### Moving around

With this, we can define the following terms:
- cycles : $t$, the number times all of the conditional posteriors $P_i \in \mathcal{P}$ have been executed.
- position : the index of the current conditional posterior $P_i$ in $\mathcal{P}$
- steps : the number of cycles times the number of positions in a cycle, that is, the total number of draws from any $P_i$. It is always equal to $cycles * p + position$.

With this, we can define a few types of cycles and give some meaning to the parameter sets that come out of the process:
- full cycle: a cycle where the starting and ending positions are the same
- partial cycle: a cycle where the starting and ending positions are not the same
- perfect cycle: a cycle where the ending position is zero
- draw : a number or set of numbers drawn from $P_i$, where $i = position$. A draw where only one $P_i$ is used is called a 'step'.
- sample : some number, $d$, of draws.

Any type of cycle is also a type of sample. Thus, a full perfect sample is one drawn from a cycle that starts and ends at 0. The easiest way to get $s$ samples from the Gibbs sampler is to simply make $s$ full perfect cycles and record the results. 

#### Using the sampler

Thus, let us call our sampler `G`. At its start, it has a position of $0$, and has taken $0$ steps. If the user runs `G.step()`. it will compute a draw of $P_0$ and increment the position by one. If we have more than one $P_i \in \mathcal{P}$, it will take a partial perfect cycle to get a full sample. 


By default, the current parameter values, the most recent draws are stored by default in the sampler state, `G._state`. The set of most recent draws, either in the current cycle or in the previous cycle, is called the *front* of the sampler, or the front sample. That is, its draws constitute a full imperfect sample. 

The *current* sample is a full perfect sample. It contains the draws that have already been taken in the current cycle, and `None` for draws in the cycle that have not occurred. Likewise, the *previous* sample is a full perfect sample from the previous cycle. 

This is important, since the Gibbs sampler uses the *front* parameters, which is just the of the *current* and *previous* parmeters.

This is actually subclass of a dictionary, and you can do anything with it that you can do with a dictionary. But, it makes its elements available in dot notation as well. So, the following are both valid ways to retrieve something from the sampler state: 
```python
G._state.Betas
G._state['Betas']
```
If you try to store something that would override a dict property or attribute (like `update` or `clear`), a `TypeError` is raised. 

So, I'm using the following terminology to talk about the sampler:
- steps: the 
- cycles
- position
- sample

In [ ]:
dh.sample(cycles=1000)